In [1]:
# Install required packages to connect to google sheets
!pip install gspread oauth2client df2gspread
!pip install geopy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 98 kB 1.5 MB/s eta 0:00:011
     |████████████████████████████████| 89 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 984 kB/s eta 0:00:01
     |████████████████████████████████| 155 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 1.1 MB/s eta 0:00:01
  Created wheel for df2gspread: filename=df2gspread-1.0.4-py3-none-any.whl size=11953 sha256=1669277371b94c95035ec3a45da92db4c366c98deea19a42a90e211992eb35f3
  Stored in directory: /home/aravind/.cache/pip/wheels/4d/78/20/cf9a2810f93885ad71bd351c8b604455ba160ac4b0e9b7d526
Successfully built df2gspread
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-

In [3]:
#Load gsheets libraries
import gspread 
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#authenticate gsheets
google_key_file = 'service_key.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

In [4]:
spreadsheet_key = '1auoQ9XanosnM7RUInzqeZi9EIgwtCtmtubNpXrfF6OM' 
wks_name = 'twitter_data'


book = gc.open_by_key(spreadsheet_key) 
worksheet = book.worksheet(wks_name) 

In [5]:
# get and shows all values from spreadsheet
table = worksheet.get_all_values()
table

[]

In [6]:
# get data to test writing into file 
import seaborn as sns
tips = sns.load_dataset('tips')

In [7]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [8]:
#Write in the gsheets
d2g.upload(tips, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'twitter_data' id:1401220933>

In [15]:
# Load twitter credentials
import json
with open("covid19-sentanalysis-twitter_credentials.json") as datafile:
    data = json.load(datafile)

In [16]:
import tweepy
from textblob import TextBlob


# Step 1 - Authenticate twitter
consumer_key= data['consumer_key'] #'API_CONSUMER_KEY_HERE'
consumer_secret=  data['consumer_secret']#'CONSUMER_SECRET_HERE'

access_token= data['access_token_key'] #'ACCESS_TOKEN_HERE'
access_token_secret= data['access_token_secret'] #'ACCESS_TOKEN_SECRET_HERE'



auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)


try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

#Print 5 tweets for testing purposes - Should be deleted afterwards
home_tweets = api.home_timeline(count=5)
print("printing tweets from timeline \n ")
for tweet in home_tweets:
    print(tweet.text)
    print("")

Authentication OK
printing tweets from timeline 
 
Good 12 PM Tweepy

Good evening Tweepy

Hello Tweepy



**GET Twitter Stream and Do Sentiment Analysis in Real time**


In [30]:
from tweepy import Stream
from tweepy import StreamListener
#from google.colab import files
from  geopy.geocoders import Nominatim
from datetime import datetime
import pandas as pd
import json
import re
import csv
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/aravind/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [35]:
trump = 0
warren = 0

header_name = ['Trump', 'Warren','Text','id', 'created_at', 'user_id','coordinates','location']
with open('sentiments.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames = header_name)
    writer.writeheader()

# sentiments_df = pd.DataFrame(columns=['Trump','Warren'])
#api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

class Listener(StreamListener):
    def __init__(self):
        super().__init__()
        self.max_tweets = 10
        self.tweet_count = 0
        self.geolocator = Nominatim()
    
    def on_data(self, data):
        raw_twitts = json.loads(data)
        try:
            #  Fields we need: id, created_at, text, coordinates, author_id
            full_tweets = raw_twitts.copy()
   
            # TO DO: we must drop from full_tweets the tweets that consist only on RT, numbers, etc (see regex used below)
            
            tweets = raw_twitts['text']
            tweets = ' '.join(re.sub("(@[A-Za-z0-9]+) | ({*0-9A-Za-z \t]) |] (\wt:\/\/\St+)", " ", tweets).split())
            tweets = ' '.join(re.sub('RT',' ', tweets).split())  

            #tweets = raw_twitts['text']
            #tweets = ' '.join(re.sub("(@[A-Za-z0-9]+) | ({*0-9A-Za-z \t]) |] (\wt:\/\/\St+)", " ", tweets).split())
            #tweets = ' '.join(re.sub('RT',' ', tweets).split())
            blob = TextBlob(tweets.strip())
            global trump
            global warren

            trump_sentiment = 0
            warren_sentiment = 0

            for sent in blob.sentences:
                if "Trump" in sent and "Warren" not in sent:
                    trump_sentiment = trump_sentiment + sent.sentiment.polarity
                else:
                    warren_sentiment = warren_sentiment + sent.sentiment.polarity

            trump = trump + trump_sentiment
            warren = warren + warren_sentiment
            
            #get timestamp from created_at
            time_created_at = raw_twitts['created_at']
            t = time_created_at.split('+0000 ')
            time = t[0] +t[1]
            format_time = '%a %b %d %H:%M:%S %Y'
            date_time = datetime.strptime(time,format_time)
            ts = int(date_time.timestamp())
            
            #get lat, long from location
            lat = None
            long = None
            if raw_twitts['user']['location']:
                loc = self.geolocator.geocode(raw_twitts['user']['location'])
                if loc:
                    lat = loc.latitude
                    long = loc.longitude    
            
            if lat and long:
                with open('sentiments.csv', 'a') as file:
                    writer = csv.DictWriter(file, fieldnames=header_name)
                    info = {'id':raw_twitts['id'],
                            'user_id':raw_twitts['user']['id'], 
                            'Text':raw_twitts['text'],
                            'created_at':raw_twitts['created_at'],
                            'timestamp':ts,
                            'location':raw_twitts['user']['location'],
                            'latitude':lat,
                            'longitude':long,
                            'Trump': trump,
                            'Warren': warren                            
                           }
                    writer.writerow(info)

            # sentiments_df['Trump'].append(trump)
            # sentiments_df['Warren'].append(warren)
            # sentiments_df.to_csv('sentiments.csv') 
            print (tweets,'\n')    
        except:
            print('ERROR got')
        else:
            self.tweet_count+=1
        if(self.tweet_count==self.max_tweets):
            ## HERE: WRITE INTO FILE 
            #Write in the gsheets
            d2g.upload(tips, spreadsheet_key, wks_name, credentials=credentials, row_names=False)
            print("completed")
            return(False)
        else:
            decoded = json.loads(data)

    def on_error(self, status):
        print(status)

In [24]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [34]:
twitter_stream = Stream(auth, Listener())
twitter_stream.filter(track = ['Trump','Warren'])

/home/aravind/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


1590231507
None None
@DearAuntCrabby: Democrats need to sack up, stop being babies, stop the purity tests, and fuck Trump in November. 

1590231507
None None
@islandsky72: It's funny how they all forget trump almost killed Harley Davidson a couple of years ago with hi… 

1590231507
40.3275697 -80.0394977
not null
@jaketapper: Asked by why he was promoting the use of hydroxychloroquine, the president said, “If you look at the one survey, th… 

1590231507
38.4493315 -78.8688833
not null
President Trump called on churches to reopen this weekend. What do you think? #DoNotGiveUpMeetingTogether.… https://t.co/h5CFeuSZFF 

1590231507
38.8339578 -104.8253485
not null
@IngrahamAngle: Thank you, President Trump, for standing up for our essential right to worship. We need to lift up our nation in prayer… 

1590231507
51.937336599999995 4.315671601132608
not null
Donald Trump is a very dangerous precedent. Sorry. 

1590231507
None None
@RyanAFournier: BREAKING: The Michigan Attorney General just s

KeyboardInterrupt: 